<a href="https://colab.research.google.com/github/akshatshahd/Machine-Learning/blob/main/9.perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import kagglehub

# Step 1: Download dataset
path = kagglehub.dataset_download("mathchi/diabetes-data-set")
print("Path to dataset files:", path)

# Step 2: Load CSV file (usually diabetes.csv)
csv_path = path + "/diabetes.csv"
df = pd.read_csv(csv_path)

print("Dataset sample:\n", df.head())

# Step 3: Prepare inputs (X) and target (y)
X = df.drop(columns=['Outcome']).values  # Features
y = df['Outcome'].values.reshape(-1, 1)  # Binary target

# Step 4: One-hot encode the target (binary classification)
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y)

# Step 5: Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_onehot, test_size=0.2, random_state=42)

# Step 7: MLP code

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class MLP:
    def __init__(self, input_size, hidden_size, output_size, lr=0.01):
        self.lr = lr
        self.W1 = np.random.uniform(-1, 1, (input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.uniform(-1, 1, (hidden_size, output_size))
        self.b2 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        error_output = y - output
        delta_output = error_output * sigmoid_derivative(output)

        error_hidden = delta_output.dot(self.W2.T)
        delta_hidden = error_hidden * sigmoid_derivative(self.a1)

        W2_update = self.a1.T.dot(delta_output)
        W1_update = X.T.dot(delta_hidden)

        self.W2 += self.lr * W2_update
        self.b2 += self.lr * np.sum(delta_output, axis=0, keepdims=True)
        self.W1 += self.lr * W1_update
        self.b1 += self.lr * np.sum(delta_hidden, axis=0, keepdims=True)

        # Print weight change magnitudes
        print(f"||ΔW1|| = {np.linalg.norm(self.lr * W1_update):.6f}, ||ΔW2|| = {np.linalg.norm(self.lr * W2_update):.6f}")

    def train(self, X, y, epochs=300):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if (epoch + 1) % 50 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.6f}")

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)

# Step 8: Initialize and train MLP

input_size = X_train.shape[1]  # 8 features in this dataset
hidden_size = 8
output_size = y_train.shape[1]  # 2 classes (binary)

mlp = MLP(input_size, hidden_size, output_size, lr=0.01)
mlp.train(X_train, y_train, epochs=300)

# Step 9: Evaluate

predictions = mlp.predict(X_test)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_test_labels)

print(f"\nTest Accuracy: {accuracy * 100:.2f}%")


Using Colab cache for faster access to the 'diabetes-data-set' dataset.
Path to dataset files: /kaggle/input/diabetes-data-set
Dataset sample:
    Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
||ΔW1|| = 0.153613, ||ΔW2|| = 0.679173
||ΔW1|| = 0.148207, ||ΔW2|| = 0.140683
||ΔW1|| = 0.139575, ||ΔW2|| = 0.103721
||ΔW1|| = 0.132504, ||ΔW2|| = 0.097774
||Δ